# Code to calculate desired evaluation metrics from best run timeseries

## import library and define functions

In [2]:
import numpy as np
import yaml
import os
import warnings
import json
import hydroeval as he
import pandas as pd
from tqdm import tqdm
import hydroeval as he

In [3]:
def calculate_nse(modeled_data, observed_data):
    mean_observed = np.mean(observed_data)
    numerator = np.sum(np.power(observed_data - modeled_data, 2))
    denominator = np.sum(np.power(observed_data - mean_observed, 2))
    return 1 - np.divide(numerator, denominator)

def load_basin_list(basin_filename):
    with open(basin_filename, 'r') as file:
        lines = file.readlines()
        # Remove leading/trailing whitespaces and newline characters
        lines = [line.strip() for line in lines]
    basin_list_str = lines
    return basin_list_str

def load_time_index(time_split_file):
    with open(time_split_file) as f:
        time_split = json.load(f)
        print(time_split)
        start_time = time_split['calibration']['start_datetime']
        end_time = time_split['calibration']['end_datetime']
        hourly_index = pd.date_range(start=start_time, end=end_time, freq='H')
    return hourly_index, start_time, end_time

def load_best_run_timeseries(best_runs_dir, basin_id, hourly_index): # Get the file names in the directory
    best_runs_files = os.listdir(best_runs_dir)
    best_run_for_a_basin = [file_name for file_name in best_runs_files if file_name.startswith(basin_id)]
    if len(best_run_for_a_basin)==1:
        with open(os.path.join(best_runs_dir, best_run_for_a_basin[0])) as f:
            best_run = json.load(f)
    elif len(best_run_for_a_basin)==0:
        warnings.warn("No calibration was done for this basin")
    elif len(best_run_for_a_basin)>1:
        warnings.warn("Multiple calibration runs are mixed up in one folder, check")
    best_run_timeseries = pd.DataFrame(best_run['best simulation results'], columns=['simulated'])
    best_run_timeseries.set_index(hourly_index, inplace=True)
    return best_run_timeseries
    
def load_obs_timeseries(obs_data_dir, start_time, end_time, basin_id):
    obs_data_files = os.listdir(obs_data_dir)
    obs_data_file = [file_name for file_name in obs_data_files if file_name.startswith(basin_id)]

    if len(obs_data_file)==1:
        obs_data_ = pd.read_csv(os.path.join(obs_data_dir, obs_data_file[0]))
    else:
        warnings.warn("data is gone!")
        
    obs_data_['date'] = pd.to_datetime(obs_data_['date'])
    obs_data_.set_index(obs_data_['date'], inplace=True)
    obs_data = obs_data_[start_time:end_time]
    
    return obs_data

    

## Read configs

In [4]:
# Read the config file
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Access the config variables
results_dir = config['io_dir']['results_dir'].replace("${cwd}", "..")
best_runs_dir = os.path.join(results_dir, 'best_runs')
basin_filename = config['model_settings']['basin_file'].replace("${cwd}", "..")
time_split_file = config['model_settings']['time_split_file'].replace("${cwd}", "..")
obs_data_dir = config['io_dir']['usgs_streamflow_dir'].replace("${cwd}", "..")

## Loop through basins and evaluate

In [7]:
# Initialize
basin_list_str = load_basin_list(basin_filename)
df_results = pd.DataFrame(columns=['BasinID', 'NSE', 'NNSE', 'logNSE', 'KGE', 'logKGE'])
plot_results = False

# # To test one basin
# i=0
# basin_id = basin_list_str[i]
# for basin_id in ([basin_list_str[i]]):

# Loop through all the basins
for basin_id in tqdm(basin_list_str):

    # Load time split
    hourly_index, start_time, end_time = load_time_index(time_split_file)
    
    # Load and combine obs & simulated data 
    best_run_timeseries = load_best_run_timeseries(best_runs_dir=best_runs_dir, basin_id=basin_id, hourly_index=hourly_index)
    obs_data = load_obs_timeseries(obs_data_dir=obs_data_dir, start_time=start_time, end_time=end_time, basin_id=basin_id)
    df_eval_sim = obs_data.join(best_run_timeseries)
    
    # If you want to plot results
    if plot_results: 
        print(df_eval_sim.head())
        df_eval_sim[['QObs(mm/h)', 'simulated']].plot()
        
    # Get desired columns
    modeled_data_ = df_eval_sim['simulated'].values
    observed_data_ = df_eval_sim['QObs(mm/h)'].values
    
    # Skip nan values in the observed data
    modeled_data = modeled_data_[~np.isnan(observed_data_)]
    observed_data = observed_data_[~np.isnan(observed_data_)]
    
    # Evalute (you can add as many eval metrics here)
    nse = calculate_nse(modeled_data=modeled_data, observed_data=observed_data)
    nnse = 1/(2-nse)
    lognse = calculate_nse(modeled_data=np.log(modeled_data), observed_data=np.log(observed_data))
    kge = he.evaluator(he.kge, simulations=modeled_data, evaluations=observed_data)
    logkge = he.evaluator(he.kge, simulations=np.log(modeled_data), evaluations=np.log(observed_data))
    
    df_results = pd.concat([df_results, pd.DataFrame({
        'BasinID': [basin_id], 
        'NSE': [nse], 
        'NNSE':[nnse], 
        'logNSE':[lognse],
        'KGE':[kge],
        'logKGE': [logkge]
        })])

# Reset the index of the DataFrame
df_results = df_results.reset_index(drop=True)

# Print the results DataFrame
print(df_results)
df_results.to_csv(os.path.join(best_runs_dir, 'best_runs_eval.csv'))

{'spinup-for-calibration': {'start_datetime': '1998-10-01 00:00:00', 'end_datetime': '1999-09-30 23:00:00', 'note': 'Used for CFE model spin-up by 2023 team (1yr before calibration period)'}, 'calibration': {'start_datetime': '1999-10-01 00:00:00', 'end_datetime': '2008-09-30 23:00:00', 'note': 'Used for calibrating CFE model & training LSTM by 2023 team. Based on previous studies (Frame et al., 2022; Kratzert et al., 2019; Hoedt et al., 2021; Kratzert et al., 2021)'}, 'spinup-for-testing': {'start_datetime': '2008-10-01 00:00:00', 'end_datetime': '2009-09-30 23:00:00', 'note': 'Used for CFE model spin-up by 2023 team (1yr before testing period)'}, 'testing': {'start_datetime': '2009-10-01 00:00:00', 'end_datetime': '2010-09-30 23:00:00', 'note': "Used for checking CFE & LSTM performance after calibration by 2023 team. No validation (hyper-parameter tuning) performed for LSTM. 1990 doesn't have much data"}}


ValueError: Length mismatch: Expected 52608 rows, received array of length 78912